# FICO challenge dataset

Dataset from FICO challenge downloadable here:


2018 competition was won by Rudin et al and here's their model results
http://dukedatasciencefico.cs.duke.edu/models/

## 1: Data Prep and feature engineering
- Normal Scaling of numeric variables (prep_option = 1)
- Binning (following Rudin) and one hot encoding (prep_option = 2)
- Binning and applying WOE, calculating WOE on Rudin's bins (prep_option = 3)
- Binning and applying WOE, following Rudin (prep_option = 4)



In [1]:
prep_option = 2
save_d = True

In [2]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [3]:
cwd = os.getcwd()
print(cwd)
os.chdir(cwd)

C:\Users\brizio\Documents\PythonNB\FICOchallenge


In [4]:
RESULT_PATH = './results'

if not os.path.exists(RESULT_PATH):
        os.mkdir(RESULT_PATH)

## Read in original dataset

In [5]:
data = pd.read_csv("Data/heloc_dataset_v1.csv")
print(data.shape)
data.head()

(10459, 24)


,RiskPerformance,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,...,PercentInstallTrades,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance
0,Bad,55,144,4,84,20,3,0,83,2,...,43,0,0,0,33,-8,8,1,1,69
1,Bad,61,58,15,41,2,4,4,100,-7,...,67,0,0,0,0,-8,0,-8,-8,0
2,Bad,67,66,5,24,9,0,0,100,-7,...,44,0,4,4,53,66,4,2,1,86
3,Bad,66,169,1,73,28,1,1,93,76,...,57,0,5,4,72,83,6,4,3,91
4,Bad,81,333,27,132,12,0,0,100,-7,...,25,0,1,1,51,89,3,1,0,80


In [6]:
# import pandas_profiling
# profile = pandas_profiling.ProfileReport(data, minimal=False)
# profile

In [7]:
dups = data.duplicated(subset=None, keep='first')
data[dups]['RiskPerformance'].value_counts()

Bad     323
Good    264
Name: RiskPerformance, dtype: int64

Of the 588 duplicates (all -9 values, i.e. not found at bureaux) there's a 55% bad rate, these should be treated differently given that there's no info on them but left for later.

### Split X & y

In [8]:
CLASS = 'RiskPerformance'

# Split X and y
X = data.drop(columns=[CLASS])
y = data[CLASS]
class_names = sorted(y.unique(),  reverse=True)

y.value_counts()

Bad     5459
Good    5000
Name: RiskPerformance, dtype: int64

Encode y

In [9]:
y_onehot = pd.get_dummies(y)[['Bad']]
np.array(np.unique(y_onehot, return_counts=True)).T

array([[   0, 5000],
       [   1, 5459]], dtype=int64)

### Save y

In [10]:
if prep_option == 1 and save_d == True:
    # Save y data (to save column names etc.)
    y.to_csv("./Data/y_data.csv", index=False)
    print('Y data saved')

## Option 1: Scale X

In [11]:
if prep_option == 1:
    ## Scale the Training Data Matrix

#     from sklearn.preprocessing import MinMaxScaler
#     scaler = MinMaxScaler(feature_range=(-1, 1), copy=True)

#     from sklearn.preprocessing import RobustScaler
#     scaler = RobustScaler()

    # Standard Scaler is best performing
    from sklearn.preprocessing import StandardScaler
    
    scaler = StandardScaler()
    scaler.fit(X.values)
    X1 = pd.DataFrame(scaler.transform(X.values))
    X1.columns = X.columns
    X1

### Save the transformed data

In [12]:
if prep_option == 1 and save_d == True:
    # Save modified data (to save column names etc.)
    X1.to_csv("./Data/Scaled_data.csv", index=False)
    print('Data saved')

## Option 2: Bin X following the Chen, lin, Rudin segmentations

Binning is necessary for both options 2,3 and 4

In [13]:
if prep_option in [2,3,4]:
    input_names = list(X.columns)
    input_names_bin = [x + '_bin' for x in input_names]

    X1 = pd.DataFrame()

    # ExternalRiskEstimate
    X1[input_names_bin[0]]=pd.cut(x=X[input_names[0]], 
                                          bins=[-float("inf"),-9,-8,-7,63,70,75,80,float("inf")]
                                         )
    X1[input_names_bin[0]].value_counts(sort=True)

    # TradeOpenTime
    X1[input_names_bin[1]]=pd.cut(x=X[input_names[1]], 
                                          bins=[-float("inf"),-9,-8,-7,91,134,263,float("inf")]
                                         )
    X1[input_names_bin[1]].value_counts(sort=True)


    X1[input_names_bin[2]]=pd.cut(x=X[input_names[2]], 
                                          bins=[-float("inf"),-9,-8,-7,19,float("inf")]
                                         )
    X1[input_names_bin[2]].value_counts(sort=True)

    X1[input_names_bin[3]]=pd.cut(x=X[input_names[3]], 
                                          bins=[-float("inf"),-9,-8,-7,48,69,96,float("inf")]
                                         )
    X1[input_names_bin[3]].value_counts(sort=True)

    # NumSatisfactoryTrades
    X1[input_names_bin[4]]=pd.cut(x=X[input_names[4]], 
                                          bins=[-float("inf"),-9,-8,-7,2,5,12,21,float("inf")]
                                         )
    X1[input_names_bin[4]].value_counts(sort=True)

    # TradeFrequency
    X1[input_names_bin[5]]=pd.cut(x=X[input_names[5]], 
                                          bins=[-float("inf"),-9,-8,-7,1,2,11,12,float("inf")]
                                         )
    X1[input_names_bin[5]].value_counts(sort=True)

    X1[input_names_bin[6]]=pd.cut(x=X[input_names[6]], 
                                          bins=[-float("inf"),-9,-8,-7,1,7,9,float("inf")]
                                         )
    X1[input_names_bin[6]].value_counts(sort=True)

    X1[input_names_bin[11]]=pd.cut(x=X[input_names[11]], 
                                          bins=[-float("inf"),-9,-8,-7,0,9,16,27,float("inf")]
                                         )
    X1[input_names_bin[11]].value_counts(sort=True)


    X1[input_names_bin[12]]=pd.cut(x=X[input_names[12]], 
                                          bins=[-float("inf"),-9,-8,-7,2,3,6,11,float("inf")]
                                         )
    X1[input_names_bin[12]].value_counts(sort=True)

    # Delinquency
    X1[input_names_bin[7]]=pd.cut(x=X[input_names[7]], 
                                          bins=[-float("inf"),-9,-8,-7,59,84,89,96,float("inf")]
                                         )
    X1[input_names_bin[7]].value_counts(sort=True)

    X1[input_names_bin[8]]=pd.cut(x=X[input_names[8]], 
                                          bins=[-float("inf"),-9,-8,-7,17,32,47,float("inf")]
                                         )
    X1[input_names_bin[8]].value_counts(sort=True)

    X1[input_names_bin[9]]=pd.cut(x=X[input_names[9]], 
                                          bins=[-float("inf"),-9,-8,-7,5,6,float("inf")]
                                         )
    X1[input_names_bin[9]].value_counts(sort=True)

    X1[input_names_bin[10]]=pd.cut(x=X[input_names[10]], 
                                          bins=[-float("inf"),-9,-8,-7,2,float("inf")]
                                         )
    X1[input_names_bin[10]].value_counts(sort=True)

    # Installment
    X1[input_names_bin[13]]=pd.cut(x=X[input_names[13]], 
                                          bins=[-float("inf"),-9,-8,-7,35,46,57,84,float("inf")]
                                         )
    X1[input_names_bin[13]].value_counts(sort=True)

    X1[input_names_bin[18]]=pd.cut(x=X[input_names[18]], 
                                          bins=[-float("inf"),-9,-8,-7,35,70,float("inf")]
                                         )
    X1[input_names_bin[18]].value_counts(sort=True)

    X1[input_names_bin[20]]=pd.cut(x=X[input_names[20]], 
                                          bins=[-float("inf"),-9,-8,-7,2,3,11,13,float("inf")]
                                         )
    X1[input_names_bin[20]].value_counts(sort=True)


    #Inquiry
    X1[input_names_bin[14]]=pd.cut(x=X[input_names[14]], 
                                          bins=[-float("inf"),-9,-8,-7,0,2,8,22,float("inf")]
                                         )
    X1[input_names_bin[14]].value_counts(sort=True)

    X1[input_names_bin[15]]=pd.cut(x=X[input_names[15]], 
                                          bins=[-float("inf"),-9,-8,-7,1,4,8,float("inf")]
                                         )
    X1[input_names_bin[15]].value_counts(sort=True)

    X1[input_names_bin[16]]=pd.cut(x=X[input_names[16]], 
                                          bins=[-float("inf"),-9,-8,-7,2,float("inf")]
                                         )
    X1[input_names_bin[16]].value_counts(sort=True)

    # Revolving balance
    X1[input_names_bin[17]]=pd.cut(x=X[input_names[17]], 
                                          bins=[-float("inf"),-9,-8,-7,14,37,72,float("inf")]
                                         )
    X1[input_names_bin[17]].value_counts(sort=True)

    X1[input_names_bin[19]]=pd.cut(x=X[input_names[19]], 
                                          bins=[-float("inf"),-9,-8,-7,3,4,7,11,float("inf")]
                                         )
    X1[input_names_bin[19]].value_counts(sort=True)

    # Utilisation
    X1[input_names_bin[21]]=pd.cut(x=X[input_names[21]], 
                                          bins=[-float("inf"),-9,-8,-7,1,2,3,5,float("inf")]
                                         )
    X1[input_names_bin[21]].value_counts(sort=True)

    # Trade with balance
    X1[input_names_bin[22]]=pd.cut(x=X[input_names[22]], 
                                          bins=[-float("inf"),-9,-8,-7,47,66,73,86,float("inf")]
                                         )
    X1[input_names_bin[22]].value_counts(sort=True)
    print(pd.DataFrame(X1.columns.values))

                                         0
0                 ExternalRiskEstimate_bin
1                MSinceOldestTradeOpen_bin
2            MSinceMostRecentTradeOpen_bin
3                       AverageMInFile_bin
4                NumSatisfactoryTrades_bin
5          NumTrades60Ever2DerogPubRec_bin
6          NumTrades90Ever2DerogPubRec_bin
7                       NumTotalTrades_bin
8               NumTradesOpeninLast12M_bin
9               PercentTradesNeverDelq_bin
10                MSinceMostRecentDelq_bin
11            MaxDelq2PublicRecLast12M_bin
12                         MaxDelqEver_bin
13                PercentInstallTrades_bin
14            NetFractionInstallBurden_bin
15            NumInstallTradesWBalance_bin
16        MSinceMostRecentInqexcl7days_bin
17                        NumInqLast6M_bin
18               NumInqLast6Mexcl7days_bin
19          NetFractionRevolvingBurden_bin
20          NumRevolvingTradesWBalance_bin
21  NumBank2NatlTradesWHighUtilization_bin
22         

In [14]:
print(X1[X1.columns[0]].value_counts(sort=False))

X1.head()

(-inf, -9.0]     598
(-9.0, -8.0]       0
(-8.0, -7.0]       0
(-7.0, 63.0]    2178
(63.0, 70.0]    2371
(70.0, 75.0]    1618
(75.0, 80.0]    1383
(80.0, inf]     2311
Name: ExternalRiskEstimate_bin, dtype: int64


,ExternalRiskEstimate_bin,MSinceOldestTradeOpen_bin,MSinceMostRecentTradeOpen_bin,AverageMInFile_bin,NumSatisfactoryTrades_bin,NumTrades60Ever2DerogPubRec_bin,NumTrades90Ever2DerogPubRec_bin,NumTotalTrades_bin,NumTradesOpeninLast12M_bin,PercentTradesNeverDelq_bin,...,PercentInstallTrades_bin,NetFractionInstallBurden_bin,NumInstallTradesWBalance_bin,MSinceMostRecentInqexcl7days_bin,NumInqLast6M_bin,NumInqLast6Mexcl7days_bin,NetFractionRevolvingBurden_bin,NumRevolvingTradesWBalance_bin,NumBank2NatlTradesWHighUtilization_bin,PercentTradesWBalance_bin
0,"(-7.0, 63.0]","(134.0, 263.0]","(-7.0, 19.0]","(69.0, 96.0]","(12.0, 21.0]","(2.0, 11.0]","(-7.0, 1.0]","(16.0, 27.0]","(-7.0, 2.0]","(59.0, 84.0]",...,"(35.0, 46.0]","(-9.0, -8.0]","(-7.0, 2.0]","(-7.0, 0.0]","(-7.0, 1.0]","(-7.0, 2.0]","(14.0, 37.0]","(7.0, 11.0]","(-7.0, 1.0]","(66.0, 73.0]"
1,"(-7.0, 63.0]","(-7.0, 91.0]","(-7.0, 19.0]","(-7.0, 48.0]","(-7.0, 2.0]","(2.0, 11.0]","(1.0, 7.0]","(0.0, 9.0]","(-7.0, 2.0]","(96.0, inf]",...,"(57.0, 84.0]","(-9.0, -8.0]","(-9.0, -8.0]","(-7.0, 0.0]","(-7.0, 1.0]","(-7.0, 2.0]","(-7.0, 14.0]","(-7.0, 3.0]","(-9.0, -8.0]","(-7.0, 47.0]"
2,"(63.0, 70.0]","(-7.0, 91.0]","(-7.0, 19.0]","(-7.0, 48.0]","(5.0, 12.0]","(-7.0, 1.0]","(-7.0, 1.0]","(0.0, 9.0]","(3.0, 6.0]","(96.0, inf]",...,"(35.0, 46.0]","(35.0, 70.0]","(-7.0, 2.0]","(-7.0, 0.0]","(1.0, 4.0]","(2.0, inf]","(37.0, 72.0]","(3.0, 4.0]","(-7.0, 1.0]","(73.0, 86.0]"
3,"(63.0, 70.0]","(134.0, 263.0]","(-7.0, 19.0]","(69.0, 96.0]","(21.0, inf]","(-7.0, 1.0]","(-7.0, 1.0]","(27.0, inf]","(2.0, 3.0]","(89.0, 96.0]",...,"(46.0, 57.0]","(70.0, inf]","(3.0, 11.0]","(-7.0, 0.0]","(4.0, 8.0]","(2.0, inf]","(37.0, 72.0]","(4.0, 7.0]","(2.0, 3.0]","(86.0, inf]"
4,"(80.0, inf]","(263.0, inf]","(19.0, inf]","(96.0, inf]","(5.0, 12.0]","(-7.0, 1.0]","(-7.0, 1.0]","(9.0, 16.0]","(-7.0, 2.0]","(96.0, inf]",...,"(-7.0, 35.0]","(70.0, inf]","(-7.0, 2.0]","(-7.0, 0.0]","(-7.0, 1.0]","(-7.0, 2.0]","(37.0, 72.0]","(-7.0, 3.0]","(-7.0, 1.0]","(73.0, 86.0]"


#### Encode binned variables

In [25]:
# One-hot
if prep_option == 2:
    X_onehot = pd.get_dummies(X1)
    
    def drop_cardinality_1(df):
            res = df
            for col in df.columns:
                    if len(df[col].unique()) == 1:
                            res = res.drop(col,axis=1)
            return res
        
    X_onehot = drop_cardinality_1(X_onehot)
    
    print(X_onehot.shape)

(10459, 130)


#### Save the transformed data

In [26]:
if prep_option == 2 and save_d == True:
    os.chdir(cwd)
    # Save modified data (to save column names etc.)
    X_onehot.to_csv("./Data/Bin_Encoded_data_v2.csv", index=False)
    print('Data saved')
#     y.to_csv("./Data/y_data.csv", index=False)

Data saved


## Input WoE like in Rudin et al 

#### Option 3: calculate WOE from scratch on the given bins

In [17]:
def WOE_calc(df, y_onehot):
    
    df2 = df.copy()
    df2['Y'] = y_onehot
    
    WOE_df = pd.DataFrame()
    for x in tqdm(df.columns.values):
#         print(x)
#         print(df2.groupby(x).count().Y.to_frame().reset_index())
        d3 = df2.groupby(x).count().Y.to_frame().reset_index()
        d3['EVENT'] = df2.groupby(x).sum().reset_index()['Y']
        
        d3.insert(loc=0, column='Feature', value=np.repeat(x,len(df2.groupby(x).count().Y))  )    
        d3.columns = ['Feature', 'Bin', 'Count', 'Event']
        
        d3['Non_Event'] = d3['Count'] - d3['Event']
        d3["Event_Rate"] = d3['Event']/d3['Count']
        d3["Non_Event_Rate"] = d3['Non_Event']/d3['Count']

        d3["DIST_EVENT"] = d3['Event']/d3.groupby('Feature').sum()['Event'].to_numpy()
        d3["DIST_NON_EVENT"] = d3['Non_Event']/d3.groupby('Feature').sum()['Non_Event'].to_numpy()
        
        d3['Feat'] = d3['Feature'].astype(str) +"_"+d3['Bin'].astype(str) 
        
        d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
        d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)

#         print(d3)

        WOE_df = WOE_df.append(d3)
            
    return WOE_df

if prep_option == 3:
    WOE_df = WOE_calc(X1, y_onehot)
    WOE_df
    

In [18]:
## Save the bins and WOE calculated
# WOE_df.to_csv("WOE_bins.csv", index=False)

#### Option 4: load Rudin's WOE from WOE_bins_Rud.csv

In [19]:
if prep_option == 4:
    csv = "WOE_bins_Rud.csv"
    # csv = "synth_nonlinear.csv"
    WOE_df = pd.read_csv(csv)
    print(csv,"loaded")

### Apply WOE to data

In [20]:
def WOE_apply(df, woe_df, feat_var, bin_var, woe_var):
    d5 = woe_df[[feat_var, bin_var,woe_var]]
    d5 = d5.replace([np.inf], 1.).replace([-np.inf], -1.)
    # var = 'NumTrades60Ever2DerogPubRec_bin'
    transform_suffix = 'WOE'

    for var in tqdm(X1.columns.values):

    #     print(var)
        small_df = d5[woe_df[feat_var] == var]
        transform_dict = dict(zip(small_df[bin_var],small_df[woe_var]))
        replace_cmd1 = ''
    #     print(transform_dict)

        for i in sorted(transform_dict.items()):
    #         print(i)
            replace_cmd1 = replace_cmd1 + str(i[1]) + str(' if str(x) == "') + str(i[0]) + '" else '
        replace_cmd1 = replace_cmd1 + '0'
    #     print(replace_cmd1)
        if replace_cmd1 != '0':
            try:
    #             print(df2[var].astype(str).apply(lambda x: eval(replace_cmd1)))
                df[var + '_' +transform_suffix] = df[var].astype(str).apply(lambda x: eval(replace_cmd1))
            except:
                'something wrong'
    return df



In [21]:
if prep_option == 3:
    WOE_var = 'WOE'
if prep_option == 4:
    WOE_var = 'WOE_Rud'

if prep_option in [3,4]:
    df_woe = WOE_apply(X1, WOE_df, 'Feature', 'Bin', WOE_var)
    df_woe = df_woe.drop([col for col in df_woe.columns if 'WOE' not in col],axis=1)
    df_woe

## Save the transformed data

In [22]:
os.chdir(cwd)
if save_d == True and prep_option in [3,4]:
    # Save modified data (to save column names etc.)
        outcsv = "./Data/{}_data.csv".format(WOE_var, 'str')
        df_woe.to_csv(outcsv, index=False)
        print('Saved',outcsv)
